In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os
import json

print(os.listdir("../input"))
print(os.listdir("../input/imaterialist-fashion-2019-FGVC6"))

In [ ]:
data = pd.read_csv("../input/imaterialist-fashion-2019-FGVC6/train.csv")

In [ ]:
data['Category'] = data['ClassId'].apply(lambda x: int(x.split("_")[0]))
data['Attributes'] = data['ClassId'].apply(lambda x: "_".join(x.split("_")[1:]))

In [ ]:
img_names = data.ImageId.unique()

In [ ]:
train, test, _, _ = train_test_split(img_names, img_names, test_size=0.21, random_state=42)

In [ ]:
data['dataset'] = 'train'
data.loc[data['ImageId'].isin(test), 'dataset'] = 'test'

In [ ]:
data['Area'] = data.apply(lambda x: sum([int(a) for i, a in enumerate(x['EncodedPixels'].split()) if i % 2 ==1]), axis = 1)

In [ ]:
_ = data[data['dataset']=='train'].boxplot(column=['Area'])

In [ ]:
print('Min area:', data[data['dataset']=='train'].Area.min())
print('Max area:', data[data['dataset']=='train'].Area.max())

In [ ]:
area_percentile = {}

In [ ]:
for i in range(0, int(data[data['dataset']=='train'].Area.quantile(0.75)), 1000):
    area_percentile[i] = data[(data['dataset']=='train')&
                               (data['Area']<=i)].shape[0] / data[data['dataset']=='train'].shape[0]

In [ ]:
lists = sorted(area_percentile.items()) # sorted by key, return a list of tuples

x, y = zip(*lists) # unpack a list of pairs into two tuples

plt.plot(x, y)
plt.xlabel('Area')
plt.ylabel('% of objects in train dataset')
plt.show()

In [ ]:
small_lists = [x for x in lists if x[1] < 0.4]

x, y = zip(*small_lists) # unpack a list of pairs into two tuples

plt.plot(x, y)
plt.xlabel('Area')
plt.ylabel('% of objects in train dataset')
plt.show()

In [ ]:
print('% of objects with area > 10 000')
print(data[(data['dataset']=='train')& (data['Area']>=10000)].shape[0] / data[(data['dataset']=='train')].shape[0])

In [ ]:
print('Total number of train images:', data[(data['dataset']=='train')].ImageId.nunique())
print('Number of train images with large object on them:', data[(data['dataset']=='train')& (data['Area']>=10000)].ImageId.nunique())
print('Number of train images without atributes:', data[(data['dataset']=='train')&(data['Attributes']=='')].ImageId.nunique())
print('Number of train images with large object on them and without atributes:', data[(data['dataset']=='train')&(data['Area']>=1000)&(data['Attributes']=='')].ImageId.nunique())

In [ ]:
print('Total number of objects in train set:', len(data[(data['dataset']=='train')]))
print('Total number of objects in truncated train set:', len(data[(data['dataset']=='train')&(data['Area']>=1000)&(data['Attributes']=='')]))
    

In [ ]:
data_truncated = pd.concat([data[(data['dataset']=='train')&(data['Area']>=1000)&(data['Attributes']=='')],
                               data[data['dataset']=='test']])

In [ ]:
print('Nubmer of objects in original dataset: ', len(data))
print('Nubmer of objects in truncated dataset: ', len(data_truncated))


In [ ]:
data_truncated.to_csv('train_df_truncated.csv', index=False)

In [ ]:
groupby_category = data[(data['dataset']=='train')].groupby('Category')['ImageId'].count()
groupby_category.index = map(int, groupby_category.index)
groupby_category = groupby_category.sort_index()
groupby_category[:5]

fig = plt.figure(figsize=(10, 4))
x = groupby_category.index
y = groupby_category.values

sns.barplot(x,y)
plt.title("Number of images by category", fontsize=20)
plt.xlabel("Category", fontsize=20)
plt.ylabel("# of images", fontsize=20)
plt.show()